In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

windows = True # If on mac put false, windows true

In [2]:
if windows == True:
    baselines = pd.read_csv(r"C:\Users\timst\Onedrive - Nexus365\4YP\python\_storage\baselines_met.csv", index_col=0).rename(columns={"Q in single(µm3/s)": "baseline Q"}) # FOR WINDOWS
else:
    baselines = pd.read_csv(r"/Users/Debs/OneDrive - Nexus365/4YP/python/_storage/baselines_met.csv", index_col=0).rename(columns={"Q in single(µm3/s)": "baseline Q"}) # FOR MAC

In [3]:
# display(baselines)
# final = baselines.drop(['Viscocity(mmHg*s)','Resistance for U','Saturation in','Pressure Drop(mmHg)','Saturation out','Saturation ave','partial pressure blood(mmHg)','tissue partials(mmHg)','dS'],axis=1)
final  = baselines.loc[:,['Name','Number','Diameter(µm)','Length(µm)','Vt(µm3)','wall thickness(µm)']]
final.at[0,'Saturation in'] = 0.94
display(final)

,Name,Number,Diameter(µm),Length(µm),Vt(µm3),wall thickness(µm),Saturation in
0,A1,1,23.97,1267.6,3.920026e+06,4.841407,0.94
1,A2,2,19.17,930.3,1.964399e+06,4.253658,NaN
2,A3,4,15.28,543.6,2.962124e+06,3.810683,NaN
3,A4,8,12.08,302.3,2.496394e+06,3.485850,NaN
4,A5,16,9.46,161.2,1.919460e+06,3.265971,NaN
5,A6,32,7.32,154.7,1.304955e+06,3.138293,NaN
6,C,64,8.00,243.9,3.486216e+05,3.172168,NaN
7,V6,32,11.51,473.9,1.577403e+05,3.433626,NaN
8,V5,16,14.53,272.3,2.551342e+05,3.730525,NaN
9,V4,8,17.79,426.6,2.524554e+05,4.092089,NaN


In [4]:


#model input params
delay_pressure_drop = 15
optimised_value_tau = 2.292929292929293
phi_min_baseline = 0.153
n=1
ratio_drop = 0.5

#loop info
no = 1001
max_time = 80

In [5]:
t = pd.Series(np.linspace(0,max_time,no))
dt = max_time/(no-1)

#### PRessure stuff
pressure_in = pd.Series(np.zeros(len(t)))
for i in range(len(t)):
    if t[i] <= delay_pressure_drop:
        pressure_in[i] = 60
    else:
        pressure_in[i] = 60 - 34.18*ratio_drop
pressure_out = pd.Series(np.zeros(len(t))) + 60 - 34.18
pressure_difference = pressure_in - pressure_out

print('Time of simulation :',max_time,'seconds')
print('Time step :',dt,'seconds')
print('Number of iterations:',no)

Time of simulation : 80 seconds
Time step : 0.08 seconds
Number of iterations: 1001


In [6]:
def total_R(baselines,phi,alpha):
    C_ = alpha*(baselines.loc[6,'Resistance for Q']/2)/phi**4  + (1-alpha)*(baselines.loc[6,'Resistance for Q']/2)
    C_6 = (C_ + baselines.loc[5,'Resistance for Q'] + baselines.loc[7,'Resistance for Q'])/2
    C_65 = (C_6 + baselines.loc[4,'Resistance for Q'] + baselines.loc[8,'Resistance for Q'])/2
    C_654 = (C_65 + baselines.loc[3,'Resistance for Q'] + baselines.loc[9,'Resistance for Q'])/2
    C_6543 = (C_654 + baselines.loc[2,'Resistance for Q'] + baselines.loc[10,'Resistance for Q'])/2
    C_65432 = (C_6543 + baselines.loc[1,'Resistance for Q'] + baselines.loc[11,'Resistance for Q'])/2
    C_654321 = C_65432 + baselines.loc[0,'Resistance for Q'] + baselines.loc[12,'Resistance for Q']
    R_total = C_654321
    return R_total
 
def euler(alpha,delay,baselines):
    delay_number = round(delay/dt)
    
    R_tot = pd.Series(np.zeros(len(t)))
    Q_tot = pd.Series(np.zeros(len(t)))
    Q_norm = pd.Series(np.ones(len(t)))
    
    phi = pd.Series(np.ones(no))
    phi_min = pd.Series(np.ones(no))
    dphidt = pd.Series(np.zeros(no))
    
    for i in range(len(t)):
        
        if i >=delay_number: 
            R_tot[i] = total_R(baselines,phi[i],alpha)
            Q_tot[i] = pressure_difference[i]/R_tot[i]
            Q_norm[i] = Q_tot[i] / baselines['baseline Q'][0]
            dphidt[i] =  (1/optimised_value_tau)*( -phi[i-delay_number] + Q_norm[i]*(1-phi_min[i]) + phi_min[i] )
            phi[i+1] = dphidt[i]*dt + phi[i]
            phi_min[i+1] = 1-alpha*(1-phi_min_baseline)*(1-Q_norm[i])**n
            
    combined = pd.DataFrame()
    combined['t'] = t
    combined['pressure_difference'] = pressure_difference
    combined['phi'] = phi
    combined['phi_min'] = phi_min
    combined['dphidt'] = dphidt
    combined['R_tot'] = R_tot
    combined['Q_tot'] = Q_tot
    combined['Q_norm'] = Q_norm
            
    return combined

In [19]:
# response = {}

# alpha = np.linspace(0,1,num=10)
# delay = np.linspace(2.99,3, num=10)

# for i in range(len(alpha)):
#     for j in range(len(delay)):
        #first is alpha second is delay
#             response[i,j] = euler(alpha[i],delay[j])

alpha = 0.5
delay = 7.62
out = euler(alpha,delay,baselines)

In [20]:
final.at[0,'Q(µm3/s)'] = out.iloc[-1].loc['Q_tot']

In [21]:
for i in range(len(final)):
    if i ==0:
        final.at[i,'Q(µm3/s)'] = out.iloc[-1].loc['Q_tot']
    else:
        if i <=  6:
            final.loc[i,'Q(µm3/s)'] = final.loc[i-1,'Q(µm3/s)']/2
        else:
            final.loc[i,'Q(µm3/s)'] = final.loc[i-1,'Q(µm3/s)']*2

In [22]:
final

,Name,Number,Diameter(µm),Length(µm),Vt(µm3),wall thickness(µm),Saturation in,Q(µm3/s)
0,A1,1,23.97,1267.6,3.920026e+06,4.841407,0.94,1.930002e+06
1,A2,2,19.17,930.3,1.964399e+06,4.253658,NaN,9.650009e+05
2,A3,4,15.28,543.6,2.962124e+06,3.810683,NaN,4.825005e+05
3,A4,8,12.08,302.3,2.496394e+06,3.485850,NaN,2.412502e+05
4,A5,16,9.46,161.2,1.919460e+06,3.265971,NaN,1.206251e+05
5,A6,32,7.32,154.7,1.304955e+06,3.138293,NaN,6.031256e+04
6,C,64,8.00,243.9,3.486216e+05,3.172168,NaN,3.015628e+04
7,V6,32,11.51,473.9,1.577403e+05,3.433626,NaN,6.031256e+04
8,V5,16,14.53,272.3,2.551342e+05,3.730525,NaN,1.206251e+05
9,V4,8,17.79,426.6,2.524554e+05,4.092089,NaN,2.412502e+05


In [23]:
if windows == True:
    final.to_csv(r"C:\Users\timst\Onedrive - Nexus365\4YP\python\_storage\final_.csv") # FOR WINDOWS
else:
    final.to_csv(r"/Users/Debs/OneDrive - Nexus365/4YP/python/_storage/final_new.csv") # FOR MAC
    
#### CHANGE TO final WITHIN FOLDER (take off _new). this means i dont need to worry about changing values for later calcls unless i absolutely want to

In [24]:
final

,Name,Number,Diameter(µm),Length(µm),Vt(µm3),wall thickness(µm),Saturation in,Q(µm3/s)
0,A1,1,23.97,1267.6,3.920026e+06,4.841407,0.94,1.930002e+06
1,A2,2,19.17,930.3,1.964399e+06,4.253658,NaN,9.650009e+05
2,A3,4,15.28,543.6,2.962124e+06,3.810683,NaN,4.825005e+05
3,A4,8,12.08,302.3,2.496394e+06,3.485850,NaN,2.412502e+05
4,A5,16,9.46,161.2,1.919460e+06,3.265971,NaN,1.206251e+05
5,A6,32,7.32,154.7,1.304955e+06,3.138293,NaN,6.031256e+04
6,C,64,8.00,243.9,3.486216e+05,3.172168,NaN,3.015628e+04
7,V6,32,11.51,473.9,1.577403e+05,3.433626,NaN,6.031256e+04
8,V5,16,14.53,272.3,2.551342e+05,3.730525,NaN,1.206251e+05
9,V4,8,17.79,426.6,2.524554e+05,4.092089,NaN,2.412502e+05
